In [8]:
import json
import numpy as np

In [9]:
from datasets import Dataset

In [10]:
from transformers import AutoTokenizer

In [11]:
import os, sys
#list the current work dir
os.getcwd()

# sys.path is a list of absolute path strings
# sys.path.append('C:/Users/User/Desktop/Github/Chatbot_Web_Application')

'c:\\Users\\User\\Desktop\\Github\\Chatbot_Web_Application\\train_chatbot'

In [24]:
file_path = "Dataset/Ecommerce_FAQ_Chatbot_dataset.json"

with open(file_path, "r", encoding='utf-8') as json_file:
            data = json.load(json_file)

In [25]:
data['questions']

[{'question': 'How can I create an account?',
  'answer': "To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process."},
 {'question': 'What payment methods do you accept?',
  'answer': 'We accept major credit cards, debit cards, and PayPal as payment methods for online orders.'},
 {'question': 'How can I track my order?',
  'answer': "You can track your order by logging into your account and navigating to the 'Order History' section. There, you will find the tracking information for your shipment."},
 {'question': 'What is your return policy?',
  'answer': 'Our return policy allows you to return products within 30 days of purchase for a full refund, provided they are in their original condition and packaging. Please refer to our Returns page for detailed instructions.'},
 {'question': 'Can I cancel my order?',
  'answer': 'You can cancel your order if it has not been shipped yet. Please c

In [26]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-large", padding='left')

In [27]:
tokenizer.pad_token = tokenizer.eos_token

In [7]:
# formatted_data = []

# for item in data["questions"]:
#     context = item["question"]
#     response = item["answer"]
    
#     # Format as conversational input
#     input_text = f"User: {context}\nAssistant: {response}"
#     input_ids = tokenizer.encode(input_text, add_special_tokens=True)
    
#     formatted_data.append({"input_ids": input_ids})

In [28]:
dataset = Dataset.from_dict(data)

In [29]:
dataset

Dataset({
    features: ['questions'],
    num_rows: 110
})

In [30]:
def encode_data(data, tokenizer):
    # building dialog
    dialog_responses = [
        f"User: {question_answer['question']}\nAssistant: {question_answer['answer']}" 
        for question_answer in data['questions']
    ]


    encoded_inputs = tokenizer(
        dialog_responses,
        padding='max_length', # Pads the sequences to the maximum length in the batch.
        truncation=True, # Truncates sequences that exceed the maximum length.
        max_length=512, # Sets the maximum length of the encoded sequences.
        return_tensors='pt' # Set output to be pytorch tensor, tf for tensor flow, pt for pytorch
    )


    input_ids = encoded_inputs['input_ids']
    attention_mask = encoded_inputs['attention_mask']

    # Output format
    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': input_ids
    }

In [31]:
# encoded_dataset = dataset.map(encode_data, batched=True, batch_size=8)

encoded_dataset = dataset.map(encode_data, batched=True, batch_size=8, fn_kwargs={'tokenizer': tokenizer})

Map:   0%|          | 0/110 [00:00<?, ? examples/s]

In [32]:
encoded_dataset

Dataset({
    features: ['questions', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 110
})

In [33]:
type(encoded_dataset)

datasets.arrow_dataset.Dataset

In [34]:
encoded_dataset.to_json("Dataset/Encoded_Ecommerce_FAQ_Chatbot_dataset.json")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

810837

In [ ]:
# # example load json from datasets.arrow_dataset.Dataset object

# from datasets import load_dataset 

# encoded_dataset = load_dataset('json', data_files="Dataset/Encoded_Ecommerce_FAQ_Chatbot_dataset.json")